In [9]:
from tic_tac_toe import TicTacToe
from neural_network import NeuralNetwork

import numpy as np
import random
import math
import copy
import time
from IPython.display import clear_output

In [10]:
def init_pred_to_cell():
    #map prediction to cell coords
    pred_to_cell = {}
    for i in range(3):
        for j in range(3):
            pred_to_cell[i*3 + j] = (i, j)
    return pred_to_cell
pred_to_cell = init_pred_to_cell()

In [11]:
def init_ppn(ppn_num, layer_dims, random_weights=True):
    return [NeuralNetwork(layer_dims, random_weights) for i in range(ppn_num)]

def make_single_move(nn, ttt, verbose=False):
    pred_move = nn.predict(ttt.get_vector_repr()).reshape(1,9)
    pred_move = np.argsort(-pred_move)
    
    for pred in pred_move[0]:
        pred = pred_to_cell[pred]
        if ttt.is_valid_move(pred[0], pred[1]):
            ttt.make_move(pred[0], pred[1])
            if verbose:
                print(ttt)
            return

def play_single_game(nn1, nn2):
    ttt = TicTacToe('X')
    nn = [nn1, nn2]
    ttt.make_move(random.randint(0,2), random.randint(0,2))
    curr_nn_idx = 1
    while ttt.get_game_status()==0:
        curr_nn = nn[curr_nn_idx]
        make_single_move(curr_nn, ttt)
        curr_nn_idx = (curr_nn_idx+1) % 2
    return ttt.get_game_status()
    
def play_single_game_update_score(nn1, nn2, score, playerX, playerO):
    res = play_single_game(nn1, nn2)
    if res==1:
        score[playerX] += 1
        score[playerO] -= 1
    if res==2:
        score[playerX] -= 1
        score[playerO] += 1

def play_all_opponents(nn):
    '''
    nn: an array of nn to play against each other
    '''
    score = [0 for i in range(len(nn))]
    for i in range(len(nn)-1):
        for j in range(i+1 , len(nn)):
            play_single_game_update_score(nn[i], nn[j], score, i, j)
            play_single_game_update_score(nn[j], nn[i], score, j, i)
    return score


In [12]:
def random_mutate(ppn, mutate_chance_ppn, mutate_chance_indiv):
    for idx_indiv in range(2, len(ppn)):
        if random.random() < mutate_chance_ppn:
            random_mutate_indiv(ppn[idx_indiv], mutate_chance_indiv)
    return ppn

def random_mutate_indiv(indiv, mutate_chance_indiv):
    for weight in indiv.weights:
        for row in range(weight.shape[0]):
            for col in range(weight.shape[1]):
                if random.random() < mutate_chance_indiv:
                    sd = max(weight[row, col]/2, 0.1)
                    weight[row, col] += random.gauss(0, sd)
    for bias in indiv.bias:
        for row in range(bias.shape[0]):
            if random.random() < mutate_chance_indiv:
                sd = max(bias[row]/2, 0.1)
                bias[row] += random.gauss(0, sd)
    return indiv


In [13]:
def crossover_remainder(ppn, target_ppn_total):
    import copy
    new_ppn = []
    num_indiv_to_generate = target_ppn_total - len(ppn)
    
    while len(new_ppn) < num_indiv_to_generate:
        male_idx = random.randint(0, len(ppn)-1)
        female_idx = random.randint(0, len(ppn)-1)
        if male_idx != female_idx:
            num_hidden_nodes = ppn[0].layers_num_nodes[1]

            num_to_crossover = random.randint(1,num_hidden_nodes-1)

            male_idx_to_crossover = random.sample(range(num_hidden_nodes), num_to_crossover)
            female_idx_to_crossover = random.sample(range(num_hidden_nodes), num_to_crossover)

            child_nn = copy.deepcopy(ppn[male_idx])
            female_nn = copy.deepcopy(ppn[female_idx])
            for i in range(len(male_idx_to_crossover)):
                curr_child_idx_to_crossover = male_idx_to_crossover[i]
                curr_female_idx_to_crossover = female_idx_to_crossover[i]
                child_nn.weights[0][curr_child_idx_to_crossover] = female_nn.weights[0][curr_female_idx_to_crossover]
                child_nn.bias[0][curr_child_idx_to_crossover] = female_nn.bias[0][curr_female_idx_to_crossover]
                child_nn.weights[1][:,curr_child_idx_to_crossover] = female_nn.weights[1][:,curr_female_idx_to_crossover]
            new_ppn.append(child_nn)
    return new_ppn


In [14]:
def choose_top_n_percent(ppn, idx_top, survival_chance_top):
    num_survive = math.floor(len(ppn) * survival_chance_top)
    idx_survive = idx_top[:num_survive]
    return [ppn[idx] for idx in idx_survive]

def random_select_for_survival(ppn, idx_top, survival_chance_top, survival_chance_remainder):
    num_survive_top = math.floor(len(ppn) * survival_chance_top)
    idx_remainder = idx_top[num_survive_top:]
    selected_for_survival = []
    for idx in idx_remainder:
        if survival_chance_remainder > random.random():
            selected_for_survival.append(ppn[idx])
    return selected_for_survival

def random_generate_new(ppn, new_ppn, layer_dims, num_new_generate):
    if len(ppn) - len(new_ppn)>20:
        return init_ppn(num_new_generate, layer_dims)
    else:
        return []

In [15]:
def evolve(ppn, survival_chance_top, survival_chance_remainder, layer_dims, num_new_generate, mutate_chance_ppn, mutate_chance_indiv, verbose=False):
    ppn = copy.deepcopy(ppn)
    new_ppn = []
    
    scores = play_all_opponents(ppn)
    scores_sorted_idx = np.array(scores).argsort()[-1::-1]
    
    new_ppn.extend(choose_top_n_percent(ppn, scores_sorted_idx, survival_chance_top))
    new_ppn.extend(random_select_for_survival(ppn, scores_sorted_idx, survival_chance_top, survival_chance_remainder))
    new_ppn.extend(random_generate_new(ppn, new_ppn, layer_dims, num_new_generate))
    new_ppn.extend(crossover_remainder(new_ppn, len(ppn)))
    new_ppn = random_mutate(new_ppn, mutate_chance_ppn, mutate_chance_indiv)

    return new_ppn


In [73]:
#Fitness of each network, in each evolution, is determined by a round-robin match between 40 neural networks.

ppn = init_ppn(40, [18,128,9], random_weights=True)

test_ppn = init_ppn(40, [18,128,9], random_weights=True)
for i in range(200):
    ppn = evolve(ppn, 0.1, 0.1, [18,128,9], 8, 0.4, 0.2, verbose=False)
    if i%20==0: 
        print('Evolution round {}'.format(i))
    
scores = play_all_opponents(ppn)
scores_sorted_idx = np.array(scores).argsort()[-1::-1]
best_idx = scores_sorted_idx[0]
nn_best = ppn[best_idx]

Evolution round 0
Evolution round 20
Evolution round 40
Evolution round 60
Evolution round 80
Evolution round 100
Evolution round 120
Evolution round 140
Evolution round 160
Evolution round 180


In [80]:
#Round-robin match between best neural network selected after evolution, and 49 randomly generated networks
rounb_robin_ppn = init_ppn(50, [18,128,9], random_weights=True)
rounb_robin_ppn[0] = nn_best
round_robin_res = play_all_opponents(rounb_robin_ppn)
round_robin_res_sorted_idx = np.array(round_robin_res).argsort()[-1::-1]
print('5 best scores are: {}'.format([round_robin_res[i] for i in round_robin_res_sorted_idx[0:5]]))
print('Best neural network evolved was ranked number {} with a score of'
      .format(np.where(round_robin_res_sorted_idx==0)[0][0]+1), round_robin_res[0])

5 best scores are: [31, 29, 27, 27, 25]
Best neural network evolved was ranked number 1 with a score of 31


In [127]:
#play a game against the selected neural network
ttt = TicTacToe('X')

In [131]:
#player is X, and moves first
ttt.make_move(2,1)
print(ttt)
time.sleep(2)
if ttt.get_game_status()==0: 
    clear_output()
    make_single_move(nn_best, ttt, verbose=True)

 O | X |   
-----------
 O | X |   
-----------
 X | X | O 

X won
